In [1]:
import numpy as np
import os
import pandas as pd
import glob

In [2]:
os.chdir('/storage/home/nxt5197/work/589_Machine_Learning/plots/data')
power_matrix=np.load('power_matrix.npy')
# power_matrix = np.reshape(powers,(853,120,250))
import _pickle as cPickle
power_matrix = cPickle.load( open( "power_matrix.pkl", "rb" ) )
x_train = power_matrix#[:600]
x_test = power_matrix#[600:]
np.shape(x_train)

(853, 120, 250)

In [4]:
import keras
from keras import layers
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import MaxPooling2D, UpSampling2D
from keras.models import Model
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

x_train_1 = np.reshape(x_train, (len(x_train), 120, 250, 1))
x_test_1 = np.reshape(x_test, (len(x_test), 120, 250, 1))

input_img = keras.Input(shape=(120, 250, 1))

enc_conv1 = Conv2D(12, (3, 3), activation='relu', padding='same')(input_img)
enc_pool1 = MaxPooling2D((2, 2), padding='same')(enc_conv1)
enc_conv2 = Conv2D(8, (4, 4), activation='relu', padding='same')(enc_pool1)
enc_ouput = MaxPooling2D((4, 5), padding='same')(enc_conv2)

dec_conv2 = Conv2D(8, (4, 4), activation='relu', padding='same')(enc_ouput)
dec_upsample2 = UpSampling2D((2, 2))(dec_conv2)
dec_conv3 = Conv2D(12, (3, 3), activation='relu', padding='same')(dec_upsample2)
dec_upsample3 = UpSampling2D((4, 5))(dec_conv3)
dec_output = Conv2D(1, (3, 3), activation='relu', padding='same')(dec_upsample3)

autoencoder = Model(input_img, dec_output)
my_optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5)
# autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.compile(optimizer=my_optimizer, loss='binary_crossentropy')
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 120, 250, 1)]     0         
                                                                 
 conv2d_5 (Conv2D)           (None, 120, 250, 12)      120       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 60, 125, 12)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 60, 125, 8)        1544      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 15, 25, 8)        0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 15, 25, 8)         1032

In [5]:
epochs=100
autoencoder_train = autoencoder.fit(x_train_1, x_test_1,
                                    epochs=epochs,
                                    batch_size=128,
                                    shuffle=True)#,
                                    # validation_data=(x_test_1, x_test_1))

Epoch 1/100
3/7 [===========>..................] - ETA: 5s - loss: 3.2545